# Uncertainty Quantification

Author: Ben Shealy

In this notebook, we will show you how to create regression models that provide __confidence intervals__ around their predictions. Confidence intervals provide a measure of the model's uncertainty around a prediction, which can be useful in many situations. For example, if you wanted to estimate the cost of something based on input features, you might want to make a more conservative estimate when the model provides a larger confidence interval around its prediction.

Uncertainty quantification is a huge topic in machine learning, and it is continuously evolving. Here we will describe four techniques that have a proven track record and are easy to use: quantile loss, jackknife, dropout, and conformal intervals. We will use the [Boston house prices dataset](https://scikit-learn.org/stable/datasets/toy_dataset.html#boston-dataset) to demonstrate these techniques.

## Getting Started

In addition to the usual packages, you will need the `forestci` package for jackknife and the `nonconformist` package for conformal intervals. Both of these packages can be installed via `pip`.

In [ ]:
import forestci
import matplotlib.pyplot as plt
from nonconformist.cp import IcpRegressor
from nonconformist.nc import NcFactory
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn.datasets
import sklearn.ensemble
import sklearn.model_selection
import sklearn.preprocessing
from tensorflow import keras

## Load Dataset

In [ ]:
# load boston housing prices dataset
boston = sklearn.datasets.load_boston()

# display boston dataset as dataframe
pd.DataFrame(
    data=np.c_[boston['data'], boston['target']],
    columns=list(boston['feature_names']) + ['target']
)

In [ ]:
# extract dataset, target
X = boston['data']
y = boston['target']

# normalize data
X = sklearn.preprocessing.scale(X)

# create train/test split
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.2)

# print shapes of train set and test set
print('X_train shape: (%d, %d)' % X_train.shape)
print('y_train shape: (%d,)' % y_train.shape)
print('X_test shape: (%d, %d)' % X_test.shape)
print('y_test shape: (%d,)' % y_test.shape)

## Quantile Loss (Gradient Boosting)

We can train a gradient boosting model with __quantile loss__, which allows the model to predict a quantile (percentile) of the target distribution. As a comparison, the MAE loss trains the model to predict the median target value (50-th percentile), so the MAE loss is equivalent to the quantile loss with `alpha=0.50`. Therefore, to produce predictions with confidence intervals, we will train three separate models to predict the lower bound, median, and upper bound.

In [ ]:
# initialize models
gb_map = {alpha: sklearn.ensemble.GradientBoostingRegressor(loss='quantile', alpha=alpha) for alpha in [0.025, 0.500, 0.975]}

# train models
for alpha, gb in gb_map.items():
    gb.fit(X_train, y_train)

# compute predictions with confidence intervals
y_preds = {alpha: gb.predict(X_test) for alpha, gb in gb_map.items()}

In [ ]:
# compute error bars
yerr = [
    y_preds[0.500] - y_preds[0.025],
    y_preds[0.975] - y_preds[0.500]
]

# plot expected vs predicted values
plt.figure(figsize=(8, 8))
plt.errorbar(
    x=y_test,
    y=y_preds[0.5],
    yerr=yerr,
    ecolor='tab:red', c='tab:blue', ls='', marker='o', mec='w')

vmax = max(y_test.max(), y_preds[0.5].max())
plt.plot([0, vmax], [0, vmax], 'k--', zorder=0)

plt.xlabel('Expected')
plt.ylabel('Predicted')
plt.show()

## Jackknife (Random Forest)

When using a random forest, which is an ensemble of decision trees, the jackknife and the infinitesimal jackknife can be used to obtain an unbiased estimate of the variance of the decision tree predictions. The calculations are somewhat more involved so we will use the `forestci` package to provide the variance estimates. For more information, read the [original paper](https://arxiv.org/abs/1311.4555).

In [ ]:
# train model
rf = sklearn.ensemble.RandomForestRegressor()
rf.fit(X_train, y_train)

# compute predictions
y_pred = rf.predict(X_test)

# compute variance estimate
V_IJ_U = forestci.random_forest_error(rf, X_train, X_test)

In [ ]:
# compute error bars
yerr = 2.0 * np.sqrt(V_IJ_U)

# plot expected vs predicted values
plt.figure(figsize=(8, 8))
plt.errorbar(
    x=y_test,
    y=y_pred,
    yerr=yerr,
    ecolor='tab:red', c='tab:blue', ls='', marker='o', mec='w')

vmax = max(y_test.max(), y_pred.max())
plt.plot([0, vmax], [0, vmax], 'k--', zorder=0)

plt.xlabel('Expected')
plt.ylabel('Predicted')
plt.show()

## Dropout (Neural Network)

Dropout is a widely used regularization technique for neural networks, in which a portion of randomly sampled weights in the network are disabled during each training iteration. Normally, dropout is disabled during inference, but if we enable dropout then we can construct confidence intervals by taking the distribution of several repeated predictions. For more information, read the [original paper](http://proceedings.mlr.press/v48/gal16.pdf).

In [ ]:
def build_fn():
    hidden_layer_sizes = [128, 128, 128]
    l1 = 0
    l2 = 1e-5
    p_dropout = 0.1
    optimizer = 'adam'
    loss = 'mean_squared_error'

    # create a 3-layer neural network
    x_input = keras.Input(shape=X.shape[1])

    x = x_input
    for units in hidden_layer_sizes:
        x = keras.layers.Dense(
            units=units,
            activation='relu',
            kernel_regularizer=keras.regularizers.l1_l2(l1, l2),
            bias_regularizer=keras.regularizers.l1_l2(l1, l2)
        )(x)

        if p_dropout != None:
            x = keras.layers.Dropout(p_dropout)(x, training=True)

    y_output = keras.layers.Dense(units=1)(x)

    mlp = keras.models.Model(x_input, y_output)

    # compile the model
    mlp.compile(optimizer=optimizer, loss=loss)

    return mlp



def inverse_tau(N, lmbda=1e-5, p_dropout=0.1, ls_2=0.005):
    return (2 * N * lmbda) / (1 - p_dropout) / ls_2 

In [ ]:
# initialize model
mlp = keras.wrappers.scikit_learn.KerasRegressor(
    build_fn=build_fn,
    batch_size=32,
    epochs=200,
    verbose=False,
    validation_split=0.1
)

# train model
mlp.fit(X_train, y_train)

# compute several predictions for each sample
y_preds = np.array([mlp.predict(X_test) for _ in range(10)])

# compute mean and 95% confidence interval
y_pred = np.mean(y_preds, axis=0)
y_lower = np.percentile(y_preds,  2.5, axis=0)
y_upper = np.percentile(y_preds, 97.5, axis=0)

# compute tau adjustment
tau_inv = inverse_tau(X_train.shape[0])

In [ ]:
# compute error bars
yerr = [
    (y_pred - y_lower) + 2.0 * tau_inv,
    (y_upper - y_pred) + 2.0 * tau_inv
]

# plot expected vs predicted values
plt.figure(figsize=(8, 8))
plt.errorbar(
    x=y_test,
    y=y_pred,
    yerr=yerr,
    ecolor='tab:red', c='tab:blue', ls='', marker='o', mec='w')

vmax = max(y_test.max(), y_preds.max())
plt.plot([0, vmax], [0, vmax], 'k--', zorder=0)

plt.xlabel('Expected')
plt.ylabel('Predicted')
plt.show()

## Conformal Prediction Intervals

This technique can be used with any prediction model. An important difference, however, is that conformal intervals are __prediction intervals__ rather than confidence intervals. Whereas a confidence interval provides a specific value with an interval around it, a prediction interval only provides an interval. While we plot the midpoint of each interval in this example for clarity, in reality we would need more information about the _distribution_ of the interval to obtain the predicted value.

To implement conformal intervals, we use the `nonconformist` package to wrap a regression model of our choice into a __nonconformity function__, which in turn is the basis of an __inductive conformal regressor__. This model will provide prediction intervals in lieu of single predictions. For more information, check out the [Github repository](https://github.com/donlnz/nonconformist).

In [ ]:
# create train/calibration split
X_tr, X_cal, y_tr, y_cal = sklearn.model_selection.train_test_split(X_train, y_train, test_size=0.2)

# create underlying model
model = sklearn.ensemble.RandomForestRegressor()

# create nonconformity function
nc = NcFactory.create_nc(model)

# create inductive conformal regressor
icp = IcpRegressor(nc)

# train model
icp.fit(X_tr, y_tr)

# calibrate model
icp.calibrate(X_cal, y_cal)

# compute predictions on test data (with 95% confidence)
y_pred = icp.predict(X_test, significance=0.05)

# separate predictions into lower and upper bounds
y_lower = y_pred[:, 0]
y_upper = y_pred[:, 1]

In [ ]:
# plot expected vs predicted values
plt.figure(figsize=(8, 8))
plt.errorbar(
    x=y_test,
    y=(y_lower + y_upper) / 2,
    yerr=(y_lower - y_upper) / 2,
    ecolor='tab:red', c='tab:blue', ls='', marker='o', mec='w')

vmax = max(y_test.max(), y_upper.max())
plt.plot([0, vmax], [0, vmax], 'k--', zorder=0)

plt.xlabel('Expected')
plt.ylabel('Predicted')
plt.show()